In [1]:
import os
import geopandas as gpd
import numpy as np
from glob import glob
from multiprocessing.dummy import Pool as ThreadPool

from pySnowRadar import snowradar, picklayers
np.seterr(invalid='ignore', divide='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [2]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
can = world.loc[world['name'] == 'Canada']

In [3]:
radar_path = './data/nsidc/'
radar_files = [y for x in os.walk(radar_path) for y in glob(os.path.join(x[0], '*.nc'))]
land_flag = [can['geometry'].intersects(snowradar.SnowRadar(file, l_case='meta').poly) for file in radar_files]
radar_files = [radar_files[i] for i in [i for i, x in enumerate(land_flag) if not x.bool()]]

In [21]:
def process_snowradar(file, n_snow):
    print('Processing: ' + file + '\n')
    radar_dat = snowradar.SnowRadar(radar_files[0], l_case='full')
    radar_dat.surf_bin, radar_dat.surface = radar_dat.get_surface()
    radar_bounds = radar_dat.get_bounds(5)
    radar_sub = radar_dat.data_radar[radar_bounds[1]: radar_bounds[0], :]
    radar_dat.calcpulsewidth() 
    
    b_as, b_si = np.apply_along_axis(
        picklayers.picklayers, 
        0, 
        radar_sub,
        radar_dat.n2n,
        radar_dat.dfr,
        n_snow
    )
    result_df = gpd.pd.DataFrame({'Lat': radar_dat.lat,
                                  'Lon': radar_dat.lon,
                                  'b_as': b_as, 
                                  'b_si': b_si})
    return result_df

In [22]:
def process_parallel(files, n_snow, threads=4):
    if isinstance(n_snow, float):
        args = list(zip(files,list(np.repeat(n_snow, len(files)))))
    elif len(n_snow)==len(files):
        args = list(zip(files, n_snow))
    else:
        print('Error: n_snow length bad')
        pass
    
    if (isinstance(files, list)) & (len(files)>1):
        pool = ThreadPool(threads)
        results = pool.starmap(process_snowradar, args)
        pool.close()
        pool.join()
        return results
    else:
        print('Requires list with n>1 elements')

In [23]:
density_snow = 0.300 #Density of snow on sea ice; See Alexandrov et al., 2010, The Cryosphere
perm_snow = (1 + 0.51 * density_snow) ** 3  #Density to permitivity
n_snow = np.sqrt(perm_snow) #Permitivity to refractive index
output = process_parallel(radar_files, n_snow, threads = 4)

Processing: ./data/nsidc\IRSNO1B_20160419_04_001_deconv.nc
Processing: ./data/nsidc\IRSNO1B_20160419_04_002_deconv.nc


Processing: ./data/nsidc\IRSNO1B_20160419_04_003_deconv.nc
Processing: ./data/nsidc\IRSNO1B_20160419_04_004_deconv.nc


Processing: ./data/nsidc\IRSNO1B_20160419_04_005_deconv.nc



In [25]:
merged_output = gpd.pd.concat(output)